In [33]:
# LiveIt Platform Automatic Data Cleaning Code #

# import statements
import numpy as np
import pandas as pd
import re

In [34]:
# read in the data file
# first sql file
comment_data = pd.read_csv("/storage/home/tum224/440 Proj/resume_entry_comments.csv")
resume_data = pd.read_csv("/storage/home/tum224/440 Proj/new_resume.csv", sep=';', header=0)
skilld_data = pd.read_csv("/storage/home/tum224/440 Proj/new_resume_skill.csv", sep=';', header=0)
entry_data = pd.read_csv("/storage/home/tum224/440 Proj/resume_entries.csv", sep=';', header=0)

# second sql file
pva_comments = pd.read_csv("/storage/home/tum224/440 Proj/pva/pva_comments.csv")
pva_entries = pd.read_csv("/storage/home/tum224/440 Proj/pva/resume_entries_pva.csv", sep=';', header=0)
pva_resumes = pd.read_csv("/storage/home/tum224/440 Proj/pva/resumes_pva.csv", sep=';', header=0)

In [35]:
# clean the files by removing unneccessary columns
comment_data_clean = comment_data.drop(columns=['owner_id', 'Unnamed: 5', 'Unnamed: 6'])
pva_data_clean = pva_comments.drop(columns=['is_approved'])
resume_data_clean = resume_data.drop(columns=['title', 'subline', 'dribbble_url', 'website_url', 'postsecondary', 'coverImageURL', 'sortOrder', 'facebook_url', 'linkedin_url', 'twitter_url', 'github_url', 'email', 'email_show', 'phone', 'phone_show', 'isPublishedEntries', 'isPublishedMessages', 'resume_pdf_url', 'location', 'resume_pdf_datetime'])
resume_data2_clean = pva_resumes.drop(columns=['title', 'subline', 'dribbble_url', 'website_url', 'postsecondary', 'coverImageURL', 'sortOrder', 'facebook_url', 'linkedin_url', 'twitter_url', 'github_url', 'email', 'email_show', 'phone', 'phone_show', 'isPublishedEntries', 'isPublishedMessages', 'resume_pdf_url', 'location', 'resume_pdf_datetime'])

In [36]:
# remove the NaN values where most necessary: when representing completely wmpty storage rows
# resumes
resume_data_clean = resume_data_clean[resume_data['profileImageURL'].notna()]
resume_data2_clean = resume_data2_clean[pva_resumes['profileImageURL'].notna()]

resume_data_clean = resume_data_clean.drop(columns=['profileImageURL'])
resume_data2_clean = resume_data2_clean.drop(columns=['profileImageURL'])

# entries
entry_data = entry_data[entry_data['expDescription'].notna()]
pva_entries = pva_entries[pva_entries['expDescription'].notna()]

entry_data.reset_index(inplace=True)
pva_entries.reset_index(inplace=True)

In [37]:
# function for creating more features
def createDataset(comments):

#dataset = [['comment_id', 'comment_length', 'punct_score', 'chars_per_word', 'class']]
    dataset = []

    for num in range(len(comments)):
        word_count = 0
        char_count = 0
        punct_list = [0, 0, 0]
    
        for word in comments.iloc[num].split(' '):
        
            word_count += 1
        
            for char in word:
            
                char_count += 1
            
                if char == '.':
                    punct_list[0] += 1
                elif char == '!':
                    punct_list[1] += 1
                elif char == '?':
                    punct_list[2] += 1
        chars_per_word = char_count / word_count
    
        # A high punct_score means that there are a lot of periods, not many question
        # marks or exclaimations.
        punct_score = punct_list[0] / (punct_list[1] + punct_list[2] + 0.1)
        dataset.append([len(comments.iloc[num]), punct_score, chars_per_word])
    
    return pd.DataFrame(dataset)

In [38]:
# clean the entries of html code
# establish an re compile
TAG_RE = re.compile(r'<[^>]+>')

entry_data['expDescription'] = [re.sub(TAG_RE, '', str(x)) for x in entry_data['expDescription']]
pva_entries['expDescription'] = [re.sub(TAG_RE, '', str(x)) for x in pva_entries['expDescription']]

In [39]:
# use createDataset function to get word breakdowns
# run the comments through 
comment_data_cleaner = createDataset(comment_data_clean['comment_body'])
comment_data2_cleaner = createDataset(pva_data_clean['comment_body'])

# run the entries through
entry_data_cleaner = createDataset(entry_data['expDescription'])
entry_data2_cleaner = createDataset(pva_entries['expDescription'])

# join the tables
psu_data_comb = comment_data_clean.join(comment_data_cleaner)
pva_data_comb = comment_data2_cleaner.join(pva_data_clean)
psu_data1_comb = entry_data.join(entry_data_cleaner)
pva_data1_comb = pva_entries.join(entry_data2_cleaner)

In [41]:
# clean the cleaner entry files for later export
psu_data_entry_final = psu_data1_comb.drop(columns=['expClassID', 'expLocationID', 'expYear', 'expMonth', 'expImage','expResourceIcon', 'expAttachment', 'created', 'modified'])
pva_data_entry_final = pva_data1_comb.drop(columns=['expClassID', 'expLocationID', 'expYear', 'expMonth', 'expImage','expResourceIcon', 'expAttachment', 'created', 'modified'])

psu_data_entry_final = psu_data_entry_final.rename(columns={0: 'comment_length', 1: 'punct_score', 2: 'chars_per_word'})
pva_data_entry_final = pva_data_entry_final.rename(columns={0: 'comment_length', 1: 'punct_score', 2: 'chars_per_word'})

In [42]:
# combine comments tables for output for dashboard
# resume comments
final_comments = psu_data_comb.append(pva_data_comb, ignore_index = True)
final_comments_total = final_comments.rename(columns={0: 'comment_length', 1: 'punct_score', 2: 'chars_per_word'})

# resumes
final_resume = resume_data_clean.append(resume_data2_clean, ignore_index = True)

# resume experience entries
final_experience = psu_data_entry_final.append(pva_data_entry_final, ignore_index = True)

In [60]:
# check the amount of training in both instances
final_comments_total['class'].value_counts()

1    1518
0    1205
Name: class, dtype: int64

In [44]:
# group together JUST exemplary and average comments for classifier
clusters_d1 = final_comments_total.groupby(by = 'class')
class0_d1 = clusters_d1.get_group(0)
class1_d1 = clusters_d1.get_group(1)

# narrow down to just comments
final_case0 = class0_d1['comment_body']
final_case1 = class1_d1['comment_body']

In [58]:
# output the files
final_comments_total.to_csv("/storage/home/tum224/dashboard_comments.csv")
final_resume.to_csv("/storage/home/tum224/dashboard_resumes.csv")
final_experience.to_csv("/storage/home/tum224/dashboard_experiences.csv")
final_case0.to_csv("/storage/home/tum224/average_comments.csv")
final_case1.to_csv("/storage/home/tum224/exemplary_comments.csv")